# Student Names:
Rebecca Moore


1.   Task 1 - basic CNN
2.   Task 3 - VGG16 transfer Learning

Manish Reddy


1.   Task 1 - basic CNN
2.   Task 2 - Hash Filter and error function

In [2]:
from google.colab import drive
import tensorflow as tf
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras import  models
from keras.models import Model
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

# Preprocessing on images 

In [3]:
train_data_path = '/content/drive/MyDrive/UTKFace/'
x = []
y_a = []
y_g = []
y_r = []

root_path, dirs, files = next(os.walk(train_data_path))

for f in files:
    f_items = str(f).split('_')
    if len(f_items) == 4 and int(f_items[0]) <= 116:
        image = cv2.imread(os.path.join(root_path, f))
        image = cv2.resize(image, (224, 224))
        x.append(image)
        y_a.append(int(f_items[0]) - 1)
        y_g.append(int(f_items[1]))
        y_r.append(int(f_items[2]))

y_a = to_categorical(y_a, 116)
y_g = to_categorical(y_g, 2)
y_r = to_categorical(y_r, 5)

Takes the labels for each category and puts them into an array

In [4]:
x = np.array(x)
y_a = np.array(y_a)
y_g = np.array(y_g)
y_r = np.array(y_r)

In [5]:
train_index = int(len(x)*(1 - 0.1))

x_train = x[:train_index]
y_train_a = y_a[:train_index]
y_train_g = y_g[:train_index]
y_train_r = y_r[:train_index]

x_test = x[train_index:]
y_test_a = y_a[train_index:]
y_test_g = y_g[train_index:]
y_test_r = y_r[train_index:]

print(x.shape)
print(y_a.shape)
print(y_g.shape)
print(y_r.shape)

(23705, 224, 224, 3)
(23705, 116)
(23705, 2)
(23705, 5)


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Mounting google drive because that is where the directory with the images is stored, set up mapping for gender and races

In [6]:
drive.mount('/content/drive')

data_name = '/content/drive/MyDrive/UTKFace'

image_size = (224, 224)
batch_size = 32

TT_SPLIT = 0.7
IM_WIDTH = 224
IM_HEIGHT = 224

dict_categories = {
    'race_id': {0: 'white', 1: 'black', 2: 'asian', 3: 'indian', 4: 'others'},
    'sex_id': {0: 'male',1: 'female'}}

dict_categories['sex_tmp'] = dict((g, i) for i, g in dict_categories['sex_id'].items())
dict_categories['race_tmp'] = dict((r, i) for i, r in dict_categories['race_id'].items())

Parse through the directory and image files to extract the labels and create a dataframe

In [7]:
def parse(dataset_path, ext='jpg'):
    def extract(path):
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')

            return int(age), dict_categories['sex_id'][int(gender)], dict_categories['race_id'][int(race)]
        except Exception as ex:
            return None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    records = []
    for file in files:
        info = extract(file)
        records.append(info)   
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['age', 'gender', 'race', 'file']
    df = df.dropna()
    return df

In [8]:
df = parse_data(data_name)
df.head()

,age,gender,race,file
0,56.0,male,white,/content/drive/MyDrive/UTKFace/56_0_0_20170119...
1,56.0,male,indian,/content/drive/MyDrive/UTKFace/56_0_3_20170119...
2,56.0,male,indian,/content/drive/MyDrive/UTKFace/56_0_3_20170119...
3,56.0,male,asian,/content/drive/MyDrive/UTKFace/56_0_2_20170119...
4,56.0,male,indian,/content/drive/MyDrive/UTKFace/56_0_3_20170119...


# Generates Images and splits up the processing into batches because the dataset is so large it will crash without being split

In [9]:
class UtkGenerator():
    def __init__(self, df):
        self.df = df 
    def split(self):
        p = np.random.permutation(len(self.df))
        spliter = int(len(self.df) * TT_SPLIT)
        train = p[:spliter]
        test = p[spliter:]

        spliter = int(spliter * TT_SPLIT)
        train, valid = train[:spliter], train[spliter:]
        
        self.df['sex_id'] = self.df['gender'].map(lambda sex: dict_categories['sex_tmp'][sex])
        self.df['race_id'] = self.df['race'].map(lambda race: dict_categories['race_tmp'][race])

        self.max_age = self.df['age'].max()
        
        return train, valid, test
    
    def preprocess_image(self, img_path):
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        images, ages, races, sexs = [], [], [], []
        while True:
            for idx in image_idx:
                f = self.df.iloc[idx]
                
                age = f['age']
                race = f['race_categorical']
                sex = f['sex_categorical']
                file = f['file']
                
                im = self.preprocess_image(file)
                ages.append(age / self.max_age)
                races.append(to_categorical(race, len(dict_categories['race_categorical'])))
                sexs.append(to_categorical(sex, len(dict_categories['sex_id'])))
                images.append(im)

                if len(images) >= batch_size:
                    yield np.array(images), [np.array(ages), np.array(races), np.array(sexs)]
                    images, ages, races, sexs = [], [], [], []
                    
            if not is_training:
                break
                
generator = UtkGenerator(df)
train, valid, test = generator.split()

# Load saved VGG16 model and weights from imagenet

In [10]:
base_model = VGG16(include_top=True,
            input_tensor=None,
            weights='imagenet',
            pooling= None,
            input_shape = None)

In [11]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Add layers for race, gender, and age outputs and compile 

In [12]:
base_output = base_model.output
output_a = Dense(116, activation='softmax', name='predications_age')(base_output)
output_g = Dense(2, activation='softmax', name='predications_gender')(base_output)
output_r = Dense(5, activation='softmax', name='predications_race')(base_output)

new_model = Model(inputs=base_model.input, outputs=[output_a, output_g, output_r], name='transfer_learning_vgg16_utk')

In [13]:
new_model.summary()

Model: "transfer_learning_vgg16_utk"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                       

# Train new transfer learning model set up for UTK dataset

In [14]:
batch_size=32
valid_batch_size = 32
epochs= 5
initial_lr=0.01
val_split=0.1
test_split=0.1
data_augmentation = True

In [15]:
callbacks = [ModelCheckpoint("./model_checkpoint", monitor='val_loss')]

In [ ]:
new_model.compile(
    optimizer='adam',
    loss={
          'predications_age': 'mse', 
          'predications_race': 'categorical_crossentropy', 
          'predications_gender': 'binary_crossentropy'},
      loss_weights={
          'predications_age': 4., 
          'predications_race': 1.5, 
          'predications_gender': 0.1},
      metrics={
          'predications_age': 'mae', 
          'predications_race': 'accuracy',
          'predications_gender': 'accuracy'},
    run_eagerly=True
)

train_gen = data_generator.generate_images(train, True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid, True, batch_size=valid_batch_size)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

history = new_model.fit(x_train, [y_train_a, y_train_g, y_train_r],
                            batch_size=batch_size, epochs= epochs, 
                            callbacks=callbacks, validation_data=(x_test, [y_test_a, y_test_g, y_test_r]))
for layer in new_model.layers:
    print('layer_name:{0}=====trainable:{1}'.format(layer.name, layer.trainable))
new_model.summary()

Epoch 1/5
114/667 [====>.........................] - ETA: 8:53:51 - loss: 2.4246 - predications_age_loss: 0.0085 - predications_gender_loss: 0.6909 - predications_race_loss: 1.5475 - predications_age_mae: 0.0171 - predications_gender_accuracy: 0.5414 - predications_race_accuracy: 0.4370